<a href="https://colab.research.google.com/github/shreyus3003/TensorflowExtended/blob/master/Trash_mgm_tfx_pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trash_management_systemUsing_DeepLearning TensorFlow Extended  Pipeline 

In [ ]:

!pip install "tfx>=0.21.1,<0.22" "tensorflow>=2.1,<2.2" "tensorboard>=2.1,<2.3" "pyzmq==17.0.0"

     |████████████████████████████████| 102kB 3.2MB/s 
ERROR: apache-beam 2.17.0 has requirement httplib2<=0.12.0,>=0.8, but you'll have httplib2 0.18.1 which is incompatible.
  Found existing installation: httplib2 0.12.0
    Uninstalling httplib2-0.12.0:
      Successfully uninstalled httplib2-0.12.0


In [ ]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input


%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

/usr/local/lib/python3.6/dist-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '


In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.1.1
TFX version: 0.21.4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Set up pipeline paths

In [ ]:
from pathlib import Path
_tfx_ = "/Users/shreyus/"
#home = str(Path)
_pipeline_name = 'waste_mgm_beam'

HOME=os.environ.get
#_cifar10_root = os.path.join(os.environ['HOME'], 'cifar10')
_cifar10_root = os.path.join(_tfx_, 'waste_mgm')
print(_cifar10_root)
_data_root = os.path.join(_cifar10_root, 'data')
_module_file = os.path.join(_cifar10_root, 'waste_mgm_utils.py')
_serving_model_dir = os.path.join(_cifar10_root, 'serving_model',
                                  _pipeline_name)
print(_data_root)

/Users/shreyus/waste_mgm
/Users/shreyus/waste_mgm/data


In [ ]:
_tfx_dir = "/Users/shreyus/"
_tfx_root = os.path.join(_tfx_dir, 'tfx')
print(_tfx_root)

/Users/shreyus/tfx


In [ ]:




_pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)

_metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name,
                              'metadata.db')

In [ ]:
_module_file = '/content/waste_mgm_utils.py'
_data_root = '/content/tfrecord'
examples = external_input(_data_root)

In [ ]:
pipeline_name=_pipeline_name
pipeline_root=_pipeline_root
data_root=_data_root
module_file=_module_file
serving_model_dir=_serving_model_dir
metadata_path=_metadata_path

In [ ]:
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.components import ModelValidator
import absl
import tensorflow_model_analysis as tfma

from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import ImportExampleGen
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import external_input

In [ ]:
context = InteractiveContext()

In [ ]:
examples = external_input(_data_root)

  # Brings data into the pipeline.
example_gen = ImportExampleGen(input=examples)

In [ ]:
context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-06-17T07_12_29.070703-44timhyu/ImportExampleGen/examples/1, id: 2)]
        ))

In [ ]:
# input_split = example_gen_pb2.Input(splits=[
#       example_gen_pb2.Input.Split(name='train', pattern='train.tfrecord'),
#       example_gen_pb2.Input.Split(name='eval', pattern='test.tfrecord')
#   ])

In [ ]:
#  example_gen = ImportExampleGen(input=examples, input_config=input_split)

In [ ]:
# statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

In [ ]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

/usr/local/lib/python3.6/dist-packages/tensorflow_data_validation/arrow/arrow_util.py:239: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: /tmp/tfx-interactive-2020-06-17T07_12_29.070703-44timhyu/StatisticsGen/statistics/2, id: 3)]
        ))

In [ ]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [ ]:
schema_gen = SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: /tmp/tfx-interactive-2020-06-17T07_12_29.070703-44timhyu/SchemaGen/schema/3, id: 4)]
        ))

In [ ]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'image_raw',BYTES,required,,-
'depth',INT,required,,-
'height',INT,required,,-
'label',INT,required,,-
'width',INT,required,,-


In [ ]:
example_validator = ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(type_name: ExampleAnomalies, uri: /tmp/tfx-interactive-2020-06-17T07_12_29.070703-44timhyu/ExampleValidator/anomalies/4, id: 5)]
        ))

In [ ]:
context.show(example_validator.outputs['anomalies'])

In [ ]:
transform = Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_gen.outputs['schema'],
      module_file=module_file)


In [ ]:
context.run(transform)

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-06-17T07_12_29.070703-44timhyu/Transform/transform_graph/10/.temp_path/tftransform_tmp/39089aa402ee48f6abc69909d2281765/saved_model.pb
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ExecutionResult(
    component_id: Transform
    execution_id: 10
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-06-17T07_12_29.070703-44timhyu/Transform/transform_graph/10, id: 12)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-06-17T07_12_29.070703-44timhyu/Transform/transformed_examples/10, id: 13)]
        ))

In [ ]:
transform.outputs

{'transform_graph': Channel(
    type_name: TransformGraph
    artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-06-17T07_12_29.070703-44timhyu/Transform/transform_graph/10, id: 12)]
), 'transformed_examples': Channel(
    type_name: Examples
    artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-06-17T07_12_29.070703-44timhyu/Transform/transformed_examples/10, id: 13)]
)}

In [ ]:
trainer = Trainer(
      module_file=module_file,
      examples=transform.outputs['transformed_examples'],
      schema=schema_gen.outputs['schema'],
      transform_graph=transform.outputs['transform_graph'],
      train_args=trainer_pb2.TrainArgs(num_steps=1000),
      eval_args=trainer_pb2.EvalArgs(num_steps=500))

In [ ]:
context.run(trainer)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_raw_xf (Conv2D)        (None, 382, 510, 64)      1792      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 380, 508, 32)      18464     
_________________________________________________________________
flatten_3 (Flatten)          (None, 6177280)           0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 30886405  
Total params: 30,906,661
Trainable params: 30,906,661
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tfx-interactive-2020-06-17T07_12_29.070703-44timhyu/Trainer/model/11/serving_model_dir', '_tf_random_seed': None, '_save_summary_steps': 100,

In [ ]:
import os  # used for directory operations
import tensorflow as tf
from PIL import Image  # used to read images from directory

In [ ]:
!unzip /content/drive/My\ Drive/garbage-classification.zip

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:

cwd = "/content/garbage-classification/Garbage classification/Garbage classification/"

recordPath = "/content/tfrecord/"

bestNum = 1000

num = 0

recordFileNum = 0

keys = [str(i) for i in list(range(10))]
values = [i for i in list(range(10))]

classes ={'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5} 

print(classes)

recordFileName = ("train.tfrecords-%.3d" % recordFileNum)

writer = tf.io.TFRecordWriter(recordPath + recordFileName)

In [ ]:
import cv2
print("Creating the 000 tfrecord file")
for name, label in classes.items():
    print(name)
    print(label)
    class_path = os.path.join(cwd, name)
    print(class_path)
    for img_name in os.listdir(class_path):

      img_path = os.path.join(class_path, img_name)
      image_string = open(img_path, 'rb').read()
    
      image_shape = tf.image.decode_jpeg(image_string).shape
      print(image_shape)
      


      feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
      }

      example = tf.train.Example(features=tf.train.Features(feature=feature))
      writer.write(example.SerializeToString())
writer.close()


In [ ]:
raw_image_dataset = tf.data.TFRecordDataset('/content/tfrecord/train.tfrecords-000')


image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):

  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

In [ ]:
import IPython.display as display
for image_features in parsed_image_dataset:
  image_raw = image_features['image_raw'].numpy()


  display.display(display.Image(data=image_raw))